In [1]:
import numpy as np 
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.9.0 --yes 
import folium # map rendering library
from IPython.display import display # For display map Folium in my System

#!conda install -c anaconda basemap --yes 

#!conda install -c conda-forge geocoder --yes 

import geocoder # import geocoder


print('Libraries imported.')

Libraries imported.


### After Import the librasies - Let's get our Home page with Requests and work with BS4

In [2]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
wikeHtml = bs(resp.text, 'html.parser')

### Get the table with class WikiTable

In [4]:
tableWiki = wikeHtml.find_all('table', attrs={'class': 'wikitable'})
tableWiki

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>
 <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Pa

### Now I wat all 'td' from before table

In [5]:
tdsWikeToronto = tableWiki[0].contents[1].find_all('td')
tdsWikeToronto

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights

### Creating the dataframe

In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', ] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

### Put the data into list from table before

In [7]:
listpostalCode = []
listBorough = []
listNeighborhood = []
count = 0
for data in tdsWikeToronto: 
    if(count == 0):
        listpostalCode.append(data.text)
        count += 1
        continue
    elif(count == 1):
        listBorough.append(data.text)
        count += 1
        continue
    else:
        listNeighborhood.append(data.text)
        count = 0
        continue
        

### Fill the DataFrame Pandas

In [8]:
for x in range(0, len(listBorough)):
    neighborhoods = neighborhoods.append({'PostalCode': listpostalCode[x],
                                          'Borough': listBorough[x],
                                          'Neighborhood': listNeighborhood[x]}, ignore_index=True)
neighborhoods  

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
...,...,...,...
283,M8Z,Etobicoke,Mimico NW\n
284,M8Z,Etobicoke,The Queensway West\n
285,M8Z,Etobicoke,Royal York South West\n
286,M8Z,Etobicoke,South of Bloor\n


### Dropping the values Not assigned from dataset

In [9]:
indexNeigh = neighborhoods[ neighborhoods['Borough'] == 'Not assigned' ].index
neighborhoods.drop(indexNeigh , inplace=True)
neighborhoods.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West\n
207,M8Z,Etobicoke,Mimico NW\n
208,M8Z,Etobicoke,The Queensway West\n
209,M8Z,Etobicoke,Royal York South West\n


### Grouping the previous dataframe by PostalCode and Borough

In [10]:
neigh_grouped = neighborhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).reset_index()
neigh_grouped.head(90)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge\n, Malvern\n]"
1,M1C,Scarborough,"[Highland Creek\n, Rouge Hill\n, Port Union\n]"
2,M1E,Scarborough,"[Guildwood\n, Morningside\n, West Hill\n]"
3,M1G,Scarborough,[Woburn\n]
4,M1H,Scarborough,[Cedarbrae\n]
...,...,...,...
85,M7A,Queen's Park,[Not assigned\n]
86,M7R,Mississauga,[Canada Post Gateway Processing Centre\n]
87,M7Y,East Toronto,[Business Reply Mail Processing Centre 969 Eas...
88,M8V,Etobicoke,"[Humber Bay Shores\n, Mimico South\n, New Toro..."


### Just Cleanning the data from list and change the 'Not assigned' values as requested

In [11]:
listNeigh = neigh_grouped['Neighborhood']
listBor = neigh_grouped['Borough']
for i in range(0, len(listNeigh)):
    for l in range(0, len(listNeigh[i])):       
        listNeigh[i][l] = listNeigh[i][l].replace('\n', '')
    if 'Not assigned'.strip() == listNeigh[i][0].strip() :
        listNeigh[i][0] = listBor[i]        
listNeigh

0                                       [Rouge, Malvern]
1               [Highland Creek, Rouge Hill, Port Union]
2                    [Guildwood, Morningside, West Hill]
3                                               [Woburn]
4                                            [Cedarbrae]
                             ...                        
98                                              [Weston]
99                                           [Westmount]
100    [Kingsview Village, Martin Grove Gardens, Rich...
101    [Albion Gardens, Beaumond Heights, Humbergate,...
102                                          [Northwest]
Name: Neighborhood, Length: 103, dtype: object

In [12]:
neigh_grouped['Neighborhood'] = listNeigh
neigh_grouped.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


In [13]:
neigh_grouped.tail(20)

,PostalCode,Borough,Neighborhood
83,M6R,West Toronto,"[Parkdale, Roncesvalles]"
84,M6S,West Toronto,"[Runnymede, Swansea]"
85,M7A,Queen's Park,[Queen's Park]
86,M7R,Mississauga,[Canada Post Gateway Processing Centre]
87,M7Y,East Toronto,[Business Reply Mail Processing Centre 969 Eas...
88,M8V,Etobicoke,"[Humber Bay Shores, Mimico South, New Toronto]"
89,M8W,Etobicoke,"[Alderwood, Long Branch]"
90,M8X,Etobicoke,"[The Kingsway, Montgomery Road, Old Mill North]"
91,M8Y,Etobicoke,"[Humber Bay, King's Mill Park, Kingsway Park S..."
92,M8Z,Etobicoke,"[Kingsway Park South West, Mimico NW, The Quee..."


In [14]:
neigh_grouped.shape

(103, 3)

### Now I need of the PostalCode list from DF and I will put the Lat and Long in dic Python

In [15]:
listPostalCode = neigh_grouped['PostalCode']
dicLatLong = {}

In [16]:
# Just a test with M1C PostalCode
address = 'M1C, Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of a Postal Code M1C are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of a Postal Code M1C are 43.653963, -79.387207.


### I Try to get Coordinate from Geocoder and this don't work for me

In [17]:
for postal_code in listPostalCode:   
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    if(lat_lng_coords != None):
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        dicLatLong[postal_code] = [latitude, longitude]
        print('The geograpical coordinate of a Postal Code {} are {}, {}.'.format(postal_code, latitude, longitude))

### I Try to get Coordinate from GeoPy but I don't get all. So I will use the csv.

In [31]:
for postal_code in listPostalCode:     
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
    if(location != None):
        latitude = location.latitude
        longitude = location.longitude
        dicLatLong[postal_code] = [latitude, longitude]
        print('The geograpical coordinate of a Postal Code {} are {}, {}.'.format(postal_code, latitude, longitude))


The geograpical coordinate of a Postal Code M1B are 43.653963, -79.387207.
The geograpical coordinate of a Postal Code M1C are 43.653963, -79.387207.
The geograpical coordinate of a Postal Code M1G are 43.6449033, -79.3818364.
The geograpical coordinate of a Postal Code M1W are 43.6449033, -79.3818364.
The geograpical coordinate of a Postal Code M2J are 43.6449033, -79.3818364.
The geograpical coordinate of a Postal Code M2M are 43.7859621, -79.4160307769213.
The geograpical coordinate of a Postal Code M2N are 43.6449033, -79.3818364.
The geograpical coordinate of a Postal Code M3A are 43.653963, -79.387207.
The geograpical coordinate of a Postal Code M3C are 43.7328216, -79.3469614.
The geograpical coordinate of a Postal Code M3H are 43.7561987, -79.4398017.
The geograpical coordinate of a Postal Code M3J are 43.6449033, -79.3818364.
The geograpical coordinate of a Postal Code M4X are 43.6680266, -79.3692816.
The geograpical coordinate of a Postal Code M5E are 43.6449033, -79.3818364.

### Reading the csv file

In [18]:
cord = pd.read_csv('Geospatial_Coordinates.csv')
cord.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [19]:
cord.tail(20)

,Postal Code,Latitude,Longitude
83,M6R,43.648960,-79.456325
84,M6S,43.651571,-79.484450
85,M7A,43.662301,-79.389494
86,M7R,43.636966,-79.615819
87,M7Y,43.662744,-79.321558
88,M8V,43.605647,-79.501321
89,M8W,43.602414,-79.543484
90,M8X,43.653654,-79.506944
91,M8Y,43.636258,-79.498509
92,M8Z,43.628841,-79.520999


### Put the collum Lat and Long into DF group from CSV file

In [20]:
neigh_grouped['Latitude'] = cord['Latitude']
neigh_grouped['Longitude'] = cord['Longitude']
neigh_grouped = pd.DataFrame(neigh_grouped)
neigh_grouped

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,[Weston],43.706876,-79.518188
99,M9P,Etobicoke,[Westmount],43.696319,-79.532242
100,M9R,Etobicoke,"[Kingsview Village, Martin Grove Gardens, Rich...",43.688905,-79.554724
101,M9V,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,...",43.739416,-79.588437


In [21]:
# Find a Borough cordinate to visualizate the Data
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating a Toronto map with the Lat and Long from DF and add Markes to map

In [44]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neigh_grouped in zip(neigh_grouped['Latitude'], neigh_grouped['Longitude'], neigh_grouped['Borough'], neigh_grouped['Neighborhood']):
    label = '{}, {}'.format(neigh_grouped, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Creating a Scarborough data with I will explore

In [22]:
Scarborough_data = neigh_grouped[neigh_grouped['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


### Create map of Scarborough using latitude and longitude values

In [67]:
# create map of Scarborough using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

### Foursquare login data

In [123]:
CLIENT_ID = '0N5VRHQIGR00EKNAOSEIOQUEEISSOAQIYYHMLNRB22' #Foursquare ID
CLIENT_SECRET = 'TUCKFGGFDUKNAOSEIOQUEEISSOAQIXOO1D5TS1KF' #Foursquare Secret
VERSION = '20191019' # Foursquare API version

### Foursquare query

In [124]:
#Now, let's get the top 150 venues that are in Marble Hill within a radius of 700 meters.
# type your answer here
LIMIT = 150
radius = 700
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Scarborough_data.Latitude[0], 
    Scarborough_data.Longitude[0], 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0N5VRHQIGR00EKNAOSEIOQUEEISSOAQIYYHMLNRB22&client_secret=TUCKFGGFDUKNAOSEIOQUEEISSOAQIXOO1D5TS1KF&v=20191019&ll=43.806686299999996,-79.19435340000001&radius=700&limit=150'

### Get the results from Foursquare

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dac77098ad62e002ca5f99e'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.812986306300004,
    'lng': -79.18564005302258},
   'sw': {'lat': 43.80038629369999, 'lng': -79.20306674697744}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80228301948931,
          'lng': -79.19856472801668}],
        

### Function that extracts the category of the venue

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Get the venues names and category

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Tim Hortons,Coffee Shop,43.802000,-79.198169
4,Lee Valley,Hobby Shop,43.803161,-79.199681


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


### Function to repeat the same process to all the neighborhoods in Scarborough

In [31]:
#function to repeat the same process to all the neighborhoods in Scarborough
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
scarborough_venues = getNearbyVenues(names=Scarborough_data['PostalCode'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )


M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X


### Shape of the scarborough_venues

In [34]:
print(scarborough_venues.shape)
scarborough_venues.head()

(91, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1C,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


### Lets count our data by PostalCode

In [36]:
scarborough_venues.groupby('PostalCode').count()

,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,7,7,7,7,7,7
M1G,3,3,3,3,3,3
M1H,7,7,7,7,7,7
M1J,2,2,2,2,2,2
M1K,6,6,6,6,6,6
M1L,10,10,10,10,10,10
M1M,3,3,3,3,3,3


In [37]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


### Lets separate all Venue Category into Collumns with Dummies of Pandas

In [38]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add postalCode column back to dataframe
scarborough_onehot['PostalCode'] = scarborough_venues['PostalCode'] 

# move postalCode column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,PostalCode,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
scarborough_onehot.shape

(91, 55)

### Now Let's group all venues category by PostalCode mean

In [103]:
scarborough_grouped = scarborough_onehot.groupby('PostalCode').mean().reset_index()
scarborough_grouped

,PostalCode,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,M1B,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.0,0.000000,0.000000
1,M1C,0.000000,0.000000,0.000,0.000000,0.000000,0.5,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.0,0.000000,0.000000
2,M1E,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.142857,0.000000,0.0,...,0.142857,0.000000,0.142857,0.000000,0.000,0.00,0.0,0.0,0.000000,0.000000
3,M1G,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.0,0.000000,0.000000
4,M1H,0.000000,0.142857,0.000,0.142857,0.142857,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.0,0.142857,0.000000
5,M1J,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.500000,0.000000,0.000000,0.000,0.00,0.5,0.0,0.000000,0.000000
6,M1K,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.0,0.000000,0.000000
7,M1L,0.000000,0.000000,0.000,0.200000,0.000000,0.0,0.000000,0.000000,0.2,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.1,0.000000,0.000000
8,M1M,0.333333,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0,0.0,0.000000,0.000000
9,M1N,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000,0.25,0.0,0.0,0.000000,0.000000


In [104]:
scarborough_grouped.shape

(16, 55)

### Function to sort the venues in descending order.

In [105]:
#function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = scarborough_grouped['PostalCode']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,History Museum,Bar,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,Fried Chicken Joint
3,M1G,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment
5,M1J,Smoke Shop,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Discount Store,Department Store,Bus Station,Convenience Store,Coffee Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store
7,M1L,Bakery,Bus Line,Bus Station,Soccer Field,Metro Station,Intersection,Fast Food Restaurant,Park,Vietnamese Restaurant,Convenience Store
8,M1M,American Restaurant,Motel,Movie Theater,Hakka Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Run k-means to cluster the neighborhood into 5 clusters.

In [113]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 3, 1, 4, 1, 1, 1, 2])

In [114]:
# merge neighborhoods_venues_sorted with Scarborough_data to add latitude/longitude for each PostalCode
scarborough_merged = Scarborough_data.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')
scarborough_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,History Museum,Bar,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,Fried Chicken Joint
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476,Smoke Shop,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029,Discount Store,Department Store,Bus Station,Convenience Store,Coffee Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577,Bakery,Bus Line,Bus Station,Soccer Field,Metro Station,Intersection,Fast Food Restaurant,Park,Vietnamese Restaurant,Convenience Store
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476,American Restaurant,Motel,Movie Theater,Hakka Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Just drop the last row - it is a trash, don't have data from Foursquare

In [115]:
scarborough_merged.dropna(inplace=True)

### Insert Cluster Labels into merged DataFrame

In [116]:
scarborough_merged.insert(5, 'Cluster Labels', kmeans.labels_)
scarborough_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,1,History Museum,Bar,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,1,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,Fried Chicken Joint
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,1,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476,4,Smoke Shop,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029,1,Discount Store,Department Store,Bus Station,Convenience Store,Coffee Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577,1,Bakery,Bus Line,Bus Station,Soccer Field,Metro Station,Intersection,Fast Food Restaurant,Park,Vietnamese Restaurant,Convenience Store
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476,1,American Restaurant,Motel,Movie Theater,Hakka Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848,2,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


# create map with cluster

In [118]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['PostalCode'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

In [125]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, 
                    scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


### Cluster 2

In [126]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, 
                    scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,History Museum,Bar,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Scarborough,1,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,Fried Chicken Joint
4,Scarborough,1,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store,General Entertainment
6,Scarborough,1,Discount Store,Department Store,Bus Station,Convenience Store,Coffee Shop,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Furniture / Home Store
7,Scarborough,1,Bakery,Bus Line,Bus Station,Soccer Field,Metro Station,Intersection,Fast Food Restaurant,Park,Vietnamese Restaurant,Convenience Store
8,Scarborough,1,American Restaurant,Motel,Movie Theater,Hakka Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Furniture / Home Store,Latin American Restaurant,Chinese Restaurant,Bar,Breakfast Spot,Athletics & Sports,Auto Garage
11,Scarborough,1,Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
12,Scarborough,1,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,1,Pizza Place,Rental Car Location,Noodle House,Italian Restaurant,Pharmacy,Breakfast Spot,Chinese Restaurant,Bank,Fast Food Restaurant,Thai Restaurant


### Cluster 3

In [127]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, 
                    scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,2,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Cluster 4

In [128]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, 
                    scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,Scarborough,3,Playground,Park,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Cluster 5

In [129]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, 
                    scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4,Smoke Shop,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
